<a href="https://colab.research.google.com/github/malaxovakatyunya/python-ai-Malahova-Ekaterina/blob/main/notebooks/week2b_read_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📊 Week 2: Data Analysis — Чтение и проверка данных о статуях мира

**Цель**: Научиться читать CSV-файлы из репозитория GitHub в Google Colab и выполнять базовую проверку данных с помощью pandas.

**Данные:**
- `World_Statues.csv` — информация о памятниках и статуях мира: название, тип, высота, год создания, страна и материал

**Что мы делаем:**
1. Клонируем ваш репозиторий GitHub в Colab
2. Читаем CSV-файл в pandas DataFrame
3. Очищаем и анализируем структуру столбцов
4. Просматриваем первые строки и выполняем быструю валидацию данных

## 🐱 [1] Клонируем репозиторий курса в Colab

In [7]:
# 🐱 Шаг 1. Клонируем ваш репозиторий в Colab

import os

if not os.path.exists("python-ai-Malahova-Ekaterina"):
    !git clone -q https://github.com/malaxovakatyunya/python-ai-Malahova-Ekaterina.git

%cd python-ai-Malahova-Ekaterina

print("✅ Репозиторий готов, теперь мы работаем внутри папки python-ai-Malahova-Ekaterina")

/content/python-ai-Malahova-Ekaterina/python-ai-Malahova-Ekaterina
✅ Репозиторий готов, теперь мы работаем внутри папки python-ai-Malahova-Ekaterina


## 📥 [2A] Простое чтение CSV-файлов в pandas

Сначала просто прочитаем оба CSV-файла в объекты `DataFrame`, без каких‑либо изменений.

После этого мы узнаем, сколько строк загружено в каждый датасет.

In [8]:
# 🐱 Шаг 2A. Чтение CSV-файлов в pandas

import pandas as pd

df_statues = pd.read_csv("data/World_Statues.csv")

print("✅ Загружено строк в df_cost:", len(df_statues))


✅ Загружено строк в df_cost: 1567


## 🧹 [2B] Очистка и переименование столбцов

В исходном CSV-файле есть **технические столбцы**, которые полезны для Викиданных, но мешают простому анализу:

- Столбцы `statue` и `statueType` содержат URL Wikidata (ссылки на объекты) — они не нужны для анализа.
- Столбцы `statueLabel`, `statueTypeLabel`, `countryLabel`, `materialLabel` содержат читаемые подписи (название статуи, тип, страна, материал).

В этом шаге мы:
- удалим столбцы с URL Wikidata (`statue`, `statueType`);
- переименуем столбцы с суффиксом `Label`, убрав его:
  - `statueLabel` → `statue`
  - `statueTypeLabel` → `statueType`
  - `countryLabel` → `country`
  - `materialLabel` → `material`
- приведём числовые столбцы (`height`, `creationYear`) к корректным типам:
  - `height` → `float` (высота может содержать дробную часть)
  - `creationYear` → `int` (год — целое число)

При приведении к числам мы используем:
- `pd.to_numeric(..., errors="coerce")` — преобразует значения в числа, некорректные значения превращает в `NaN`;
- `fillna(0)` — заменяет пропущенные значения (`NaN`) на 0;
- `astype(...)` — переводит столбец к нужному типу (`float` или `int`).

> ⚠️ **Важно:** этот шаг обязателен, чтобы избавиться от технических идентификаторов Викиданных и получить удобную таблицу для анализа статуй.

In [12]:
# 🧹 Шаг 2B. Очистка и переименование столбцов

import pandas as pd

# ============================================================================
# 1. ПРОВЕРКА СТРУКТУРЫ ДАННЫХ
# ============================================================================

print("📋 Исходные столбцы датасета:")
print(df_statues.columns.tolist())
print()

# Проверяем, есть ли столбцы с суффиксом "Label"
has_label_columns = any(col.endswith("Label") for col in df_statues.columns)

if has_label_columns:
    print("✅ Найдены столбцы с суффиксом 'Label' — выполняем очистку Wikidata")
else:
    print("⚠️  Столбцы с суффиксом 'Label' не найдены — данные уже очищены")
print()

# ============================================================================
# 2. УДАЛЕНИЕ СТОЛБЦОВ С URL WIKIDATA (если они существуют)
# ============================================================================

columns_to_drop = []

# Проверяем каждый столбец для удаления
for col in ["statue", "statueType"]:
    if col in df_statues.columns:
        columns_to_drop.append(col)
        print(f"🗑️  Будет удалён столбец: {col}")
    else:
        print(f"⏭️  Столбец '{col}' не найден — пропускаем")

if columns_to_drop:
    df_statues = df_statues.drop(columns=columns_to_drop)
    print(f"✅ Удалено столбцов: {len(columns_to_drop)}")
else:
    print("✅ Удалять нечего — столбцы с URL отсутствуют")
print()

# ============================================================================
# 3. ПЕРЕИМЕНОВАНИЕ СТОЛБЦОВ С СУФФИКСОМ LABEL
# ============================================================================

rename_mapping = {
    "statueLabel": "statue",
    "statueTypeLabel": "statueType",
    "countryLabel": "country",
    "materialLabel": "material"
}

# Фильтруем только существующие столбцы
actual_rename = {}
for old_name, new_name in rename_mapping.items():
    if old_name in df_statues.columns:
        actual_rename[old_name] = new_name
        print(f"✏️  Переименовываем: '{old_name}' → '{new_name}'")
    else:
        print(f"⏭️  Столбец '{old_name}' не найден — пропускаем")

if actual_rename:
    df_statues = df_statues.rename(columns=actual_rename)
    print(f"✅ Переименовано столбцов: {len(actual_rename)}")
else:
    print("✅ Переименовывать нечего — столбцы уже в финальном формате")
print()

# ============================================================================
# 4. ПРИВЕДЕНИЕ ЧИСЛОВЫХ СТОЛБЦОВ К КОРРЕКТНЫМ ТИПАМ
# ============================================================================

# Обработка столбца 'height'
if "height" in df_statues.columns:
    df_statues["height"] = pd.to_numeric(
        df_statues["height"], errors="coerce"
    ).fillna(0).astype(float)
    print("✅ Столбец 'height' приведён к типу float")
else:
    print("⚠️  Столбец 'height' не найден")

# Обработка столбца 'creationYear'
if "creationYear" in df_statues.columns:
    df_statues["creationYear"] = pd.to_numeric(
        df_statues["creationYear"], errors="coerce"
    ).fillna(0).astype(int)
    print("✅ Столбец 'creationYear' приведён к типу int")
else:
    print("⚠️  Столбец 'creationYear' не найден")
print()

# ============================================================================
# 5. ИТОГОВАЯ ПРОВЕРКА
# ============================================================================

print("="*60)
print("✅ ДАННЫЕ ОЧИЩЕНЫ И ГОТОВЫ К АНАЛИЗУ")
print("="*60)
print("\n📋 Итоговая структура датасета:")
print(df_statues.dtypes)
print()
print("🔍 Первые 5 строк:")
display(df_statues.head(5))

📋 Исходные столбцы датасета:
['statue', 'statueType', 'height', 'creationYear', 'country', 'material']

⚠️  Столбцы с суффиксом 'Label' не найдены — данные уже очищены

🗑️  Будет удалён столбец: statue
🗑️  Будет удалён столбец: statueType
✅ Удалено столбцов: 2

⏭️  Столбец 'statueLabel' не найден — пропускаем
⏭️  Столбец 'statueTypeLabel' не найден — пропускаем
⏭️  Столбец 'countryLabel' не найден — пропускаем
⏭️  Столбец 'materialLabel' не найден — пропускаем
✅ Переименовывать нечего — столбцы уже в финальном формате

✅ Столбец 'height' приведён к типу float
✅ Столбец 'creationYear' приведён к типу int

✅ ДАННЫЕ ОЧИЩЕНЫ И ГОТОВЫ К АНАЛИЗУ

📋 Итоговая структура датасета:
height          float64
creationYear      int64
country          object
material         object
dtype: object

🔍 Первые 5 строк:


,height,creationYear,country,material
0,38.18,1883,Германия,камень
1,38.18,1883,Германия,бронза
2,8.00,1920,Бразилия,железобетон
3,8.00,1920,Бразилия,талькохлорит
4,30.10,1920,Бразилия,железобетон


## 🔍 [3] Обзор данных: структура и первые строки

Сделаем короткий обзор DataFrame со статуями:

- посмотрим размер таблицы (`shape`);
- выведем список столбцов;
- посмотрим первые несколько строк;
- дополнительно посчитаем базовую статистику по высоте (`height`) и году создания (`creationYear`).

Для удобства используем функцию `show_info(df, name)`, чтобы компактно вывести информацию о датасете.

In [13]:
def show_info(df, name, n=5):
    """Краткий обзор DataFrame: имя, размер, список столбцов и первые строки."""
    print(f"\n📊 {name}")
    print("Размер:", df.shape)
    print("Столбцы:", ", ".join(df.columns))
    print("\nПервые строки:")
    print(df.head(n))

# 🔍 Шаг 3. Обзор данных

show_info(df_statues, "Статуи мира (df_statues)", n=10)  # Показываем 10 строк для лучшего обзора разнообразия

print("\n📈 Статистика по высоте (height, м):")
print(df_statues["height"].describe())

print("\n📅 Статистика по году создания (creationYear):")
print(df_statues["creationYear"].describe())

print("\n🌍 Топ-10 стран по количеству статуй:")
print(df_statues["country"].value_counts().head(10))


📊 Статуи мира (df_statues)
Размер: (1567, 4)
Столбцы: height, creationYear, country, material

Первые строки:
   height  creationYear   country      material
0   38.18          1883  Германия        камень
1   38.18          1883  Германия        бронза
2    8.00          1920  Бразилия   железобетон
3    8.00          1920  Бразилия  талькохлорит
4   30.10          1920  Бразилия   железобетон
5   30.10          1920  Бразилия  талькохлорит
6   21.00          1958  Колумбия   железобетон
7   26.00          1958  Колумбия   железобетон
8   21.00          1958  Колумбия  талькохлорит
9   26.00          1958  Колумбия  талькохлорит

📈 Статистика по высоте (height, м):
count    1567.000000
mean       61.815759
std       314.073684
min         0.150000
25%         2.570000
50%         7.000000
75%        21.000000
max      4800.000000
Name: height, dtype: float64

📅 Статистика по году создания (creationYear):
count    1567.000000
mean     1715.583918
std       600.655297
min      -489.000

## 🔬 [4] Глубокая проверка данных: эволюция материалов и национальные рекордсмены

Мы проведём два фундаментальных анализа, раскрывающих историю монументального искусства через призму данных:

### 1. Эволюция материалов по историческим эпохам
Как менялись технологии строительства статуй на протяжении двух с половиной тысячелетий?
- Разобьём все статуи на 5 исторических периодов (от античности до наших дней)
- Для каждой эпохи определим:
  - Топ-3 доминирующих материала
  - Среднюю высоту статуй для каждого материала
  - Технологический «прорыв» эпохи (например, переход от бронзы к железобетону)

> ⚠️ **Очистка данных:** перед анализом исключим аномалии (высота > 200 м). Самая высокая реальная статуя в мире — Статуя Единства (Индия) — имеет высоту 182 м. Запись с высотой 4800 м — ошибка Викиданных.

### 2. Национальные рекордсмены: самые высокие статуи мира
Какие страны создали самые грандиозные монументы?
- Для топ-15 стран по количеству статуй найдём абсолютного высотного рекордсмена
- Для каждой рекордной статуи укажем:
  - Название и тип монумента
  - Точную высоту в метрах
  - Год создания и материал
- Составим глобальный рейтинг самых высоких статуй по странам

Эти анализы превратят сырые данные в историю технологического прогресса и культурного соперничества наций в создании вечных символов.

In [15]:
# 🔬 Шаг 4. Глубокая проверка: материалы по эпохам + национальные рекордсмены

import pandas as pd

# ============================================================================
# 0. ПРОВЕРКА СТРУКТУРЫ ДАННЫХ
# ============================================================================

print("📋 Столбцы в исходном датафрейме df_statues:")
print(df_statues.columns.tolist())
print()

# Проверяем наличие всех необходимых столбцов
required_columns = ["height", "creationYear", "country"]
missing_columns = [col for col in required_columns if col not in df_statues.columns]

if missing_columns:
    print(f"❌ ОШИБКА: Отсутствуют обязательные столбцы: {missing_columns}")
    print("Проверьте, что ячейка 2B (очистка данных) выполнилась успешно!")
    raise KeyError(f"Отсутствуют столбцы: {missing_columns}")

print("✅ Все обязательные столбцы присутствуют")
print()

# ============================================================================
# 1. ПРЕДВАРИТЕЛЬНАЯ ОЧИСТКА: убираем аномалии по высоте (> 200 м)
# ============================================================================

df_clean = df_statues[df_statues["height"] <= 200].copy()
anomalies_removed = len(df_statues) - len(df_clean)
print(f"✅ Удалено аномальных записей (высота > 200 м): {anomalies_removed}")
print(f"📊 Записей для анализа: {len(df_clean)}\n")

# ============================================================================
# 2. ЭВОЛЮЦИЯ МАТЕРИАЛОВ ПО ИСТОРИЧЕСКИМ ЭПОХАМ
# ============================================================================

print("="*80)
print("1️⃣  ЭВОЛЮЦИЯ МАТЕРИАЛОВ В МОНОУМЕНТАЛЬНОМ ИСКУССТВЕ ПО ЭПОХАМ")
print("="*80)

# Функция определения эпохи по году
def get_epoch(year):
    if year < 0:
        return "Античность (до н.э.)"
    elif 0 <= year < 1500:
        return "Средневековье (0–1499)"
    elif 1500 <= year < 1800:
        return "Эпоха Просвещения (1500–1799)"
    elif 1800 <= year < 1950:
        return "Индустриальная эра (1800–1949)"
    else:
        return "Современность (1950–н.в.)"

df_clean["epoch"] = df_clean["creationYear"].apply(get_epoch)

# Порядок эпох для корректной сортировки
epoch_order = [
    "Античность (до н.э.)",
    "Средневековье (0–1499)",
    "Эпоха Просвещения (1500–1799)",
    "Индустриальная эра (1800–1949)",
    "Современность (1950–н.в.)"
]

# Анализ для каждой эпохи
for epoch_name in epoch_order:
    epoch_data = df_clean[df_clean["epoch"] == epoch_name]
    total = len(epoch_data)

    if total == 0:
        continue

    print(f"\n{'─'*80}")
    print(f"🔹 {epoch_name} | Всего статуй: {total}")
    print(f"{'─'*80}")

    # Проверяем наличие столбца 'material'
    if "material" not in epoch_data.columns:
        print("   ⚠️  Столбец 'material' отсутствует — пропускаем анализ материалов")
        continue

    # Топ-3 материала с количеством и средней высотой
    material_stats = (
        epoch_data.groupby("material")
        .agg(
            count=("material", "size"),
            avg_height=("height", "mean"),
            min_height=("height", "min"),
            max_height=("height", "max")
        )
        .sort_values("count", ascending=False)
        .head(3)
    )

    for idx, (material, row) in enumerate(material_stats.iterrows(), 1):
        print(
            f"{idx}. {material:25s} | "
            f"{int(row['count']):3d} шт. | "
            f"ср. высота: {row['avg_height']:5.1f} м | "
            f"диапазон: {row['min_height']:.1f}–{row['max_height']:.1f} м"
        )

# ============================================================================
# 3. НАЦИОНАЛЬНЫЕ РЕКОРДСМЕНЫ: САМЫЕ ВЫСОКИЕ СТАТУИ ПО СТРАНАМ
# ============================================================================

print("\n" + "="*80)
print("2️⃣  НАЦИОНАЛЬНЫЕ РЕКОРДСМЕНЫ: САМЫЕ ВЫСОКИЕ СТАТУИ МИРА")
print("="*80)

# Берём топ-15 стран по количеству статуй
top_countries = df_clean["country"].value_counts().head(15).index.tolist()

# Проверяем наличие дополнительных столбцов
has_statue = "statue" in df_clean.columns
has_statueType = "statueType" in df_clean.columns
has_material = "material" in df_clean.columns

print(f"   Столбец 'statue' присутствует: {has_statue}")
print(f"   Столбец 'statueType' присутствует: {has_statueType}")
print(f"   Столбец 'material' присутствует: {has_material}")
print()

# Собираем рекордсменов
records = []
for country in top_countries:
    country_data = df_clean[df_clean["country"] == country]
    max_idx = country_data["height"].idxmax()
    max_row = country_data.loc[max_idx]

    record = {
        "Страна": country,
        "Высота (м)": max_row["height"],
        "Год": int(max_row["creationYear"])
    }

    # Добавляем опциональные столбцы, если они есть
    if has_statue:
        record["Название"] = max_row["statue"]
    if has_statueType:
        record["Тип"] = max_row["statueType"]
    if has_material:
        record["Материал"] = max_row["material"]

    records.append(record)

df_records = pd.DataFrame(records)

# Сортируем по высоте (от самых высоких)
df_records = df_records.sort_values("Высота (м)", ascending=False).reset_index(drop=True)
df_records.index = df_records.index + 1  # Нумерация с 1

# ============================================================================
# 4. ВЫВОД ТАБЛИЦЫ РЕКОРДСМЕНОВ
# ============================================================================

print("🏆 Глобальный рейтинг самых высоких статуй по странам:")
print("   (топ-15 стран по количеству монументов)\n")

# Формируем заголовок динамически
columns_order = ["Страна"]
if has_statue:
    columns_order.append("Название")
if has_statueType:
    columns_order.append("Тип")
columns_order.extend(["Высота (м)", "Год"])
if has_material:
    columns_order.append("Материал")

# Выводим заголовок
header_parts = []
header_parts.append("Место")
header_parts.append("Страна")
if has_statue:
    header_parts.append("Название")
if has_statueType:
    header_parts.append("Тип")
header_parts.append("Высота")
header_parts.append("Год")
if has_material:
    header_parts.append("Материал")

print(" | ".join([f"{h:<15}" for h in header_parts]))
print("-" * (len(header_parts) * 17))

# Выводим данные
for idx, row in df_records.iterrows():
    parts = []
    parts.append(f"{idx}.")
    parts.append(f"{row['Страна'][:14]:<15}")

    if has_statue:
        name = row["Название"][:25] + "…" if len(row["Название"]) > 25 else row["Название"]
        parts.append(f"{name:<15}")

    if has_statueType:
        stype = row["Тип"][:15] + "…" if len(row["Тип"]) > 15 else row["Тип"]
        parts.append(f"{stype:<15}")

    parts.append(f"{row['Высота (м)']:>6.1f} м")
    parts.append(f"{row['Год']:<6}")

    if has_material:
        mat = row["Материал"][:15] + "…" if len(row["Материал"]) > 15 else row["Материал"]
        parts.append(f"{mat:<15}")

    print(" | ".join(parts))

# ============================================================================
# 5. КЛЮЧЕВЫЕ ИНСАЙТЫ
# ============================================================================

print("\n" + "="*80)
print("💡 КЛЮЧЕВЫЕ ИНСАЙТЫ АНАЛИЗА")
print("="*80)

# Инсайт 1: технологический прорыв
modern_data = df_clean[df_clean["epoch"] == "Современность (1950–н.в.)"]
if not modern_data.empty and "material" in modern_data.columns:
    concrete_count = (modern_data["material"] == "железобетон").sum()
    concrete_share = concrete_count / len(modern_data) * 100
    print(f"• Железобетон доминирует в современную эпоху: {concrete_share:.0f}% статуй после 1950 г.")

# Инсайт 2: древние технологии
ancient_data = df_clean[df_clean["epoch"] == "Античность (до н.э.)"]
if not ancient_data.empty and "material" in ancient_data.columns:
    ancient_top = ancient_data["material"].value_counts().index[0]
    ancient_avg = ancient_data["height"].mean()
    print(f"• В античности преобладал(а) {ancient_top} (ср. высота: {ancient_avg:.1f} м)")

# Инсайт 3: абсолютный рекордсмен
global_max = df_clean.loc[df_clean["height"].idxmax()]
record_name = global_max["statue"] if has_statue else "неизвестно"
record_country = global_max["country"] if "country" in global_max.index else "неизвестно"
record_material = global_max["material"] if has_material else "неизвестно"

print(f"• Абсолютный рекордсмен датасета: «{record_name}» ({record_country}, "
      f"{global_max['height']:.1f} м, {int(global_max['creationYear'])} г., {record_material})")

# Инсайт 4: культурные паттерны
if "country" in df_clean.columns and "statueType" in df_clean.columns:
    germany_data = df_clean[df_clean["country"] == "Германия"]
    if not germany_data.empty:
        germany_memorials = germany_data[germany_data["statueType"] == "военный мемориал"].shape[0]
        germany_total = len(germany_data)
        if germany_total > 0:
            print(f"• Германия: {germany_memorials} из {germany_total} статуй ({germany_memorials/germany_total*100:.0f}%) — "
                  f"военные мемориалы (культурная особенность)")

📋 Столбцы в исходном датафрейме df_statues:
['height', 'creationYear', 'country', 'material']

✅ Все обязательные столбцы присутствуют

✅ Удалено аномальных записей (высота > 200 м): 96
📊 Записей для анализа: 1471

1️⃣  ЭВОЛЮЦИЯ МАТЕРИАЛОВ В МОНОУМЕНТАЛЬНОМ ИСКУССТВЕ ПО ЭПОХАМ

────────────────────────────────────────────────────────────────────────────────
🔹 Античность (до н.э.) | Всего статуй: 3
────────────────────────────────────────────────────────────────────────────────
1. песчаник                  |   3 шт. | ср. высота: 118.0 м | диапазон: 118.0–118.0 м

────────────────────────────────────────────────────────────────────────────────
🔹 Средневековье (0–1499) | Всего статуй: 190
────────────────────────────────────────────────────────────────────────────────
1. известняк                 |  44 шт. | ср. высота:  19.5 м | диапазон: 0.9–90.0 м
2. камень                    |  40 шт. | ср. высота:  11.8 м | диапазон: 0.9–49.0 м
3. базальт                   |  36 шт. | ср. высота:   

## 📝 Summary

**Что мы сделали в этом ноутбуке (Week 2):**

- ✅ Клонировали репозиторий GitHub в Colab
- ✅ Прочитали 2 CSV-файла из `data/examples/`
- ✅ Удалили URL Wikidata и переименовали столбцы (`*Label → короткие имена`)
- ✅ Проверили структуру данных (размер, столбцы, первые строки)
- ✅ Посмотрели базовую статистику по бюджету (`capital_cost`)
- ✅ Выполнили быструю валидацию:
  - количество уникальных фильмов, стран, жанров
  - диапазоны значений
  - топ стран и жанров по числу записей

Теперь у нас есть **аккуратные, проверенные таблицы**, с которыми удобно работать дальше.

В отдельном ноутбуке для следующей недели мы будем использовать **те же данные** для:
- более сложного анализа (группировки, фильтрация),
- и построения визуализаций (графики и диаграммы). 🎨